In [1]:
import pandas as pd
import numpy as np
from nba_api.stats.static import players

import time 
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import json
import cairosvg
import statsmodels.api as sm
import base64
import requests

from nba_api.stats.static import players,teams  

import plotly.express as px

from plotly.subplots import make_subplots


def player_progression(players,start_year,end_year, category,gp=10,mpg=12):
    team_names = {
        "PHX": "Phoenix Suns",
        "DET": "Detroit Pistons",
        "POR": "Portland Trail Blazers",
        "MIL": "Milwaukee Bucks",
        "BOS": "Boston Celtics",
        "LAL": "Los Angeles Lakers",
        "ORL": "Orlando Magic",
        "NOP": "New Orleans Pelicans",
        "NOH": "New Orleans Pelicans",
        "DAL": "Dallas Mavericks",
        "DEN": "Denver Nuggets",
        "IND": "Indiana Pacers",
        "CHA": "Charlotte Hornets",
        "CHH": "Charlotte Hornets",
        "SAS": "San Antonio Spurs",
        "LAC": "Los Angeles Clippers",
        "MEM": "Memphis Grizzlies",
        "WAS": "Washington Wizards",
        "MIA": "Miami Heat",
        "ATL": "Atlanta Hawks",
        "OKC": "Oklahoma City Thunder",
        "SEA": "Seattle Supersonics",
        "UTA": "Utah Jazz",
        "NYK": "New York Knicks",
        "CHI": "Chicago Bulls",
        "MIN": "Minnesota Timberwolves",
        "NJN": "New Jersey Nets",
        "BKN": "Brooklyn Nets",
        "BRK": "Brooklyn Nets",
        "CLE": "Cleveland Cavaliers",
        "HOU": "Houston Rockets",
        "GSW": "Golden State Warriors",
        "SAC": "Sacramento Kings",
        "TOR": "Toronto Raptors",
        "PHI": "Philadelphia 76ers",
        "NOK":"New Orleans Pelicans",
        "VAN":"Vancouver Grizzlies"
    }
    
    # Normalize the data to a 0-1 range
    

    oldframe=pd.read_csv(f'https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/year_totals/{start_year}.csv',usecols=['PLAYER_ID',category,'POSS','GP','MIN'])
    newframe=pd.read_csv(f'https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/year_totals/{end_year}.csv',usecols=['PLAYER_NAME','PLAYER_ID',category,'POSS','GP','MIN'])

    oldframe['MPG']=oldframe['MIN']/oldframe['GP']
    
    newframe['MPG']=newframe['MIN']/newframe['GP']

    oldframe=oldframe[oldframe.MPG>-mpg]
    newframe=newframe[newframe.MPG>-mpg]

    oldframe=oldframe[oldframe.GP>-gp]
    newframe=newframe[newframe.GP>-gp]
    perccheck=False
    for keyword in ['freq', 'accur', 'pct', '%']:
        if keyword in category.lower():
            newframe[category]=newframe[category]*100
            oldframe[category]=oldframe[category]*100

            perccheck = True
            
    if 'rating' not in category.lower() and not perccheck and 'rate' not in category.lower():
        newframe[category]=100*newframe[category]/newframe['POSS']
        oldframe[category]=100*oldframe[category]/oldframe['POSS']

    

    oldframe.rename(columns={category:str(start_year)+'_'+category,'POSS':f'{start_year}_POSS','GP':f'{start_year}_GP','MIN':f'{start_year}_MIN'},inplace=True)
    newframe.rename(columns={category:str(end_year)+'_'+category,'POSS':f'{end_year}_POSS','GP':f'{end_year}_GP','MIN':f'{end_year}_MIN'},inplace=True)


    

    comboframe=newframe.merge(oldframe,on='PLAYER_ID',how='left')
    comboframe[category+'_diff']=comboframe[str(end_year)+'_'+category] - comboframe[str(start_year)+'_'+category] 

    category_list=[str(end_year)+'_'+category ,str(start_year)+'_'+category,category+'_diff' ]
    print(category_list)
   
    

    scaler = MinMaxScaler()
    data = pd.DataFrame(
        scaler.fit_transform(comboframe[category_list]),
        columns=['norm_'+c for c in category_list],
        index=comboframe.index
    )
    old_cols=['PLAYER_ID','PLAYER_NAME',str(end_year)+'_'+category ,str(start_year)+'_'+category,category+'_diff' ]
    data[old_cols] = comboframe[old_cols]

    player_data=data[data.PLAYER_ID.isin(players)]

    print(player_data)


    
    color_palette = px.colors.qualitative.Plotly

    # Create the figure
    fig = make_subplots(rows=1, cols=1)
    
    # Define custom colors
    grid_color = 'rgba(255, 255, 255, 0.1)'
    bg_color = '#1a1a1a'
    text_color = '#ffffff'
    
    # Plot the actual data points
    for i, cat in enumerate(category_list):
        # Plot all players data points
        fig.add_trace(go.Scatter(
            y=[cat] * len(data),
            x=data['norm_'+cat],
            mode='markers',
            marker=dict(
                size=15,
                color='#89CFF0',
                opacity=0.4,
                symbol='circle'
            ),
            hovertemplate="<b>%{text}</b><br><br>Raw: %{customdata:.2f}<extra></extra>",
            text=data['PLAYER_NAME'],
            customdata=data[cat],
            name='All Players',
            showlegend=False
        ))
        
        # Plot selected players with unique colors
        for j, player_id in enumerate(players):
            player_data = data[data.PLAYER_ID == player_id]
            fig.add_trace(go.Scatter(
                y=[cat],
                x=player_data['norm_'+cat],
                mode='markers',
                marker=dict(
                    size=20,
                    color=color_palette[j % len(color_palette)],
                    opacity=1,
                    symbol='circle'
                ),
                hovertemplate="<b>%{text}</b><br><br>Raw: %{customdata:.2f}<extra></extra>",
                text=player_data['PLAYER_NAME'],
                customdata=player_data[cat],
                name=player_data['PLAYER_NAME'].iloc[0],
                showlegend=(i==1)
            ))

    # Update layout
    fig.update_layout(
        title=dict(
            text=f'{category} Progression: {start_year} vs {end_year}',
            font=dict(size=24, color=text_color),
            x=0.5,
            y=0.95
        ),
        width=1200,
        height=800,
        showlegend=True,
        plot_bgcolor=bg_color,
        paper_bgcolor=bg_color,
        yaxis=dict(
            title=dict(text='', font=dict(size=16, color=text_color)),
            gridcolor=grid_color,
            showgrid=False,
            zeroline=True,
            zerolinecolor=grid_color,
            categoryarray=category_list,
            tickfont=dict(color=text_color, size=14),
            tickmode='array',
            tickvals=category_list,
            ticktext=[f"{end_year}",f"{start_year}",  "Difference"],
            showticklabels=True
        ),
        xaxis=dict(
            title=dict(text='Normalized Value', font=dict(size=16, color=text_color)),
            gridcolor=grid_color,
            tickfont=dict(color=text_color, size=12),
            tickangle=0,
            range=[0, 1]
        ),
        hovermode='closest',
        legend=dict(
            font=dict(color=text_color, size=12),
            bgcolor='rgba(0,0,0,0.5)',
            bordercolor=text_color,
            borderwidth=1,
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        margin=dict(t=100, b=50, l=50, r=50)
    )

    return fig
players=[1628983,2544]
fig=player_progression(players,2024,2025,'AtRimFGA')

fig.show()

['2025_AtRimFGA', '2024_AtRimFGA', 'AtRimFGA_diff']
     norm_2025_AtRimFGA  norm_2024_AtRimFGA  norm_AtRimFGA_diff  PLAYER_ID  \
472            0.425424            0.654602            0.163979       2544   
485            0.460957            0.517099            0.391413    1628983   

                 PLAYER_NAME  2025_AtRimFGA  2024_AtRimFGA  AtRimFGA_diff  
472             LeBron James       7.532716      10.294118      -2.761401  
485  Shai Gilgeous-Alexander       8.137763       8.206352      -0.068589  


NameError: name 'make_subplots' is not defined

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import plotly.express as px

def player_progression(players,start_year,end_year, category,gp=10,mpg=12):
    team_names = {
        "PHX": "Phoenix Suns",
        "DET": "Detroit Pistons",
        "POR": "Portland Trail Blazers",
        "MIL": "Milwaukee Bucks",
        "BOS": "Boston Celtics",
        "LAL": "Los Angeles Lakers",
        "ORL": "Orlando Magic",
        "NOP": "New Orleans Pelicans",
        "NOH": "New Orleans Pelicans",
        "DAL": "Dallas Mavericks",
        "DEN": "Denver Nuggets",
        "IND": "Indiana Pacers",
        "CHA": "Charlotte Hornets",
        "CHH": "Charlotte Hornets",
        "SAS": "San Antonio Spurs",
        "LAC": "Los Angeles Clippers",
        "MEM": "Memphis Grizzlies",
        "WAS": "Washington Wizards",
        "MIA": "Miami Heat",
        "ATL": "Atlanta Hawks",
        "OKC": "Oklahoma City Thunder",
        "SEA": "Seattle Supersonics",
        "UTA": "Utah Jazz",
        "NYK": "New York Knicks",
        "CHI": "Chicago Bulls",
        "MIN": "Minnesota Timberwolves",
        "NJN": "New Jersey Nets",
        "BKN": "Brooklyn Nets",
        "BRK": "Brooklyn Nets",
        "CLE": "Cleveland Cavaliers",
        "HOU": "Houston Rockets",
        "GSW": "Golden State Warriors",
        "SAC": "Sacramento Kings",
        "TOR": "Toronto Raptors",
        "PHI": "Philadelphia 76ers",
        "NOK":"New Orleans Pelicans",
        "VAN":"Vancouver Grizzlies"
    }
    
    # Normalize the data to a 0-1 range
    

    oldframe=pd.read_csv(f'https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/year_totals/{start_year}.csv',usecols=['PLAYER_ID',category,'POSS','GP','MIN'])
    newframe=pd.read_csv(f'https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/year_totals/{end_year}.csv',usecols=['PLAYER_NAME','PLAYER_ID',category,'POSS','GP','MIN'])

    oldframe['MPG']=oldframe['MIN']/oldframe['GP']
    
    newframe['MPG']=newframe['MIN']/newframe['GP']

    oldframe=oldframe[oldframe.MPG>-mpg]
    newframe=newframe[newframe.MPG>-mpg]

    oldframe=oldframe[oldframe.GP>-gp]
    newframe=newframe[newframe.GP>-gp]
    perccheck=False
    for keyword in ['freq', 'accur', 'pct', '%']:
        if keyword in category.lower():
            newframe[category]=newframe[category]*100
            oldframe[category]=oldframe[category]*100

            perccheck = True
            
    if 'rating' not in category.lower() and not perccheck and 'rate' not in category.lower():
        newframe[category]=100*newframe[category]/newframe['POSS']
        oldframe[category]=100*oldframe[category]/oldframe['POSS']

    

    oldframe.rename(columns={category:str(start_year)+'_'+category,'POSS':f'{start_year}_POSS','GP':f'{start_year}_GP','MIN':f'{start_year}_MIN'},inplace=True)
    newframe.rename(columns={category:str(end_year)+'_'+category,'POSS':f'{end_year}_POSS','GP':f'{end_year}_GP','MIN':f'{end_year}_MIN'},inplace=True)


    

    comboframe=newframe.merge(oldframe,on='PLAYER_ID',how='left')
    comboframe[category+'_diff']=comboframe[str(end_year)+'_'+category] - comboframe[str(start_year)+'_'+category] 

    category_list=[str(end_year)+'_'+category ,str(start_year)+'_'+category,category+'_diff' ]
    print(category_list)
   
    

    scaler = MinMaxScaler()
    data = pd.DataFrame(
        scaler.fit_transform(comboframe[category_list]),
        columns=['norm_'+c for c in category_list],
        index=comboframe.index
    )
    old_cols=['PLAYER_ID','PLAYER_NAME',str(end_year)+'_'+category ,str(start_year)+'_'+category,category+'_diff' ]
    data[old_cols] = comboframe[old_cols]

    player_data=data[data.PLAYER_ID.isin(players)]

    print(player_data)


    
    color_palette = px.colors.qualitative.Plotly

    # Create the figure with two x-axes
    fig = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])
    
    # Define custom colors
    grid_color = 'rgba(255, 255, 255, 0.1)'
    bg_color = '#1a1a1a'
    text_color = '#ffffff'
    color_palette = px.colors.qualitative.Prism
    
    # Plot the actual data points
    for i, cat in enumerate(category_list[:2]):  # Only plot start and end year
        # Plot all players data points
        fig.add_trace(go.Scatter(
            y=[cat] * len(data),
            x=data[cat],
            mode='markers',
            marker=dict(
                size=15,
                color='rgba(135, 206, 250, 0.4)',  # Light blue with transparency
                symbol='circle'
            ),
            hovertemplate="<b>%{text}</b><br><br>Value: %{x:.2f}<extra></extra>",
            text=data['PLAYER_NAME'],
            name='All Players',
            showlegend=False
        ), secondary_y=False)
        
        # Plot selected players with unique colors
        for j, player_id in enumerate(players):
            player_data = data[data.PLAYER_ID == player_id]
            fig.add_trace(go.Scatter(
                y=[cat],
                x=player_data[cat],
                mode='markers',
                marker=dict(
                    size=20,
                    color=color_palette[j % len(color_palette)],
                    symbol='circle'
                ),
                hovertemplate="<b>%{text}</b><br><br>Value: %{x:.2f}<extra></extra>",
                text=player_data['PLAYER_NAME'],
                name=player_data['PLAYER_NAME'].iloc[0],
                showlegend=(i==0)
            ), secondary_y=False)

    # Plot difference on secondary y-axis
    fig.add_trace(go.Scatter(
        y=[category+'_diff'] * len(data),
        x=data[category+'_diff'],
        mode='markers',
        marker=dict(
            size=15,
            color='rgba(255, 165, 0, 0.4)',  # Orange with transparency
            symbol='circle'
        ),
        hovertemplate="<b>%{text}</b><br><br>Difference: %{x:.2f}<extra></extra>",
        text=data['PLAYER_NAME'],
        name='All Players (Diff)',
        showlegend=False
    ), secondary_y=True)

    # Plot selected players' differences
    for j, player_id in enumerate(players):
        player_data = data[data.PLAYER_ID == player_id]
        fig.add_trace(go.Scatter(
            y=[category+'_diff'],
            x=player_data[category+'_diff'],
            mode='markers',
            marker=dict(
                size=20,
                color=color_palette[j % len(color_palette)],
                symbol='diamond'
            ),
            hovertemplate="<b>%{text}</b><br><br>Difference: %{x:.2f}<extra></extra>",
            text=player_data['PLAYER_NAME'],
            name=player_data['PLAYER_NAME'].iloc[0] + ' (Diff)',
            showlegend=False
        ), secondary_y=True)

    # Update layout
    fig.update_layout(
        title=dict(
            text=f'{category} Progression: {start_year} vs {end_year}',
            font=dict(size=24, color=text_color),
            x=0.5,
            y=0.98
        ),
        width=1200,
        height=800,
        showlegend=True,
        plot_bgcolor=bg_color,
        paper_bgcolor=bg_color,
        yaxis=dict(
            title=dict(text='', font=dict(size=16, color=text_color)),
            gridcolor=grid_color,
            showgrid=False,
            zeroline=True,
            zerolinecolor=grid_color,
            categoryarray=category_list,
            tickfont=dict(color=text_color, size=14),
            tickmode='array',
            tickvals=category_list[:2] + [category+'_diff'],
            ticktext=[f"{end_year}", f"{start_year}", "Difference"],
            showticklabels=True
        ),
        xaxis=dict(
            title=dict(text=category, font=dict(size=16, color=text_color)),
            gridcolor=grid_color,
            tickfont=dict(color=text_color, size=12),
            tickangle=0,
            side='bottom'
        ),
        xaxis2=dict(
            title=dict(text='Difference', font=dict(size=16, color=text_color)),
            gridcolor=grid_color,
            tickfont=dict(color=text_color, size=12),
            tickangle=0,
            side='top',
            overlaying='x',
            anchor='y'
        ),
        hovermode='closest',
        legend=dict(
            font=dict(color=text_color, size=12),
            bgcolor='rgba(0,0,0,0.5)',
            bordercolor=text_color,
            borderwidth=1,
            orientation="h",
            yanchor="bottom",
            y=1.0,
            xanchor="right",
            x=1
        ),
        margin=dict(t=100, b=50, l=50, r=50)
    )

    # Update y-axes
    fig.update_yaxes(secondary_y=False, showgrid=False)
    fig.update_yaxes(secondary_y=True, showgrid=False, overlaying='y', side='right')

    return fig
fig=player_progression(players,2024,2025,'AtRimFGA')
fig.show()